<a href="https://colab.research.google.com/github/pedrov718/Content-Based-Book-Rec-System/blob/main/book_reccomendations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [192]:
import pandas as pd
import spacy.cli
spacy.cli.download("en_core_web_lg")

import re
import string
import nltk
import numpy as np

from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from google.colab import data_table
data_table.enable_dataframe_formatter()

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [4]:
df  = pd.read_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs.csv')

In [5]:
df.head()

,ISBN,Title,Author,Year,Publisher,Blurb
0,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o..."
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea..."
2,0399135782,The Kitchen God's Wife,Amy Tan,1991,Putnam Pub Group,Winnie and Helen have kept each others worst s...
3,0425176428,What If?: The World's Foremost Military Histor...,Robert Cowley,2000,Berkley Publishing Group,Historians and inquisitive laymen alike love t...
4,1881320189,Goodbye to the Buttermilk Sky,Julia Oliver,1994,River City Pub,This highly praised first novel by fiction wri...


In [6]:
df['Blurb'][10:15]

10    Newly graduated from college, Sam Altman is gr...
11    Three passengers are dead. Fifty-six are injur...
12    In an Arizona desert, a man wanders in a daze,...
13    Dr. Ransom is abducted by a megalomaniacal phy...
14    A novel of startling scope and ambition, , dep...
Name: Blurb, dtype: object

In [193]:
# Load spacy
nlp = spacy.load('en_core_web_lg')

def clean_string(text, stem="None"):

    final_string = ""

    # Make lower
    text = text.lower()

    # Remove line breaks
    text = re.sub(r'\n', '', text)

    # Remove puncuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)

    #Remove stop words
    text = text.split()

    #Remove numbers
    text_filtered = [re.sub(r'\w*\d\w*', '', word) for word in text]

    # Stem or Lemmatize
    if stem == 'Stem':
        stemmer = PorterStemmer() 
        text_stemmed = [stemmer.stem(y) for y in text_filtered]
    elif stem == 'Lem':
        lem = WordNetLemmatizer()
        text_stemmed = [lem.lemmatize(y) for y in text_filtered]
    elif stem == 'Spacy':
        text_filtered = nlp(' '.join(text_filtered))
        text_stemmed = [y.lemma_ for y in text_filtered]
    else:
        text_stemmed = text_filtered

    final_string = ' '.join(text_stemmed)

    return final_string


In [194]:
print("Cleaning train data...\n")

df["blurb_clean"] = df['Blurb'].apply(lambda x: clean_string(x, stem='Spacy'))

Cleaning train data...



In [208]:
df.head(2)

,ISBN,Title,Author,Year,Publisher,Blurb,blurb_clean,Title_lower,blurb_clean_title
0,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,"Here, for the first time in paperback, is an o...",here for the first time in paperback be an out...,decision in normandy,decision in normandy here for the first time i...
1,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,"The fascinating, true story of the world's dea...",the fascinating true story of the world deadly...,flu: the story of the great influenza pandemic...,flu: the story of the great influenza pandemic...


In [196]:
df['Title_lower'] = df['Title'].apply(lambda x: x.lower())

In [197]:
df['blurb_clean_title'] =  df['Title_lower'] +  ' ' + df['blurb_clean']

In [198]:
df['blurb_clean_title'][10]

'wild animus newly graduate from college sam altman be grip by an inexplicable urge to lose himself in the wilderness and team up with an enigmatic young woman who seem bent on help he realize his dream'

In [230]:
df.to_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs_clean.csv')

In [231]:
df = pd.read_csv('/content/drive/MyDrive/Book Rec/books_with_blurbs_clean.csv')

#### Now that we have the data cleaned and saved to Drive we can use tfidf to convert our text data to numerical

In [232]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [233]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['blurb_clean_title'])

In [235]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index])

In [236]:
most_similiar_book()

Write a line from your favorite book? pizzq

 Title: 
 Decision in Normandy 

 Description: 
 Here, for the first time in paperback, is an outstanding military history that offers a dramatic new perspective on the Allied campaign that began with the invasion of the D-Day beaches of Normandy. Nationa advertising in Military History.


#### TOP N Number of book reccomendations

In [237]:
def top_n_similiar_books():
  
  n = int(input("How many books would you like reccomended? Please input a number. "))

  sentence = [input("Write the title or line from your favorite book? ")]
  
  y = vectorizer.transform(sentence)
  
  cosine_sim = cosine_similarity(X,y)

  # Get the pairwsie similarity scores
  sim_scores = list(enumerate(cosine_sim))
  # Sort the books based on the similarity scores
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  # Get the scores for n most similar books
  sim_scores = sim_scores[0:n]
  
  titles = []
  author = []
  desc = []

  for pair in sim_scores:
    indx, score = pair
    titles.append(df["Title"][indx])
    author.append(df["Author"][indx])
    desc.append(df["Blurb"][indx])
  
  reccs = pd.DataFrame(list(zip(titles, author, desc)), columns =['titles', 'author', 'blurbs'])
  
  return reccs

In [238]:
top_n_similiar_books()

How many books would you like reccomended? Please input a number. 4
Write the title or line from your favorite book? pizza


,titles,author,blurbs
0,The Pizza Book: Everything There Is To Know Ab...,EVELYNE SLOMON,This young chef makes pizza making easy and fu...
1,Who Put the Pizza in the Vcr: Laughing Your Wa...,Martha Bolton,In Who Put the Pizza in the VCR? Martha descri...
2,Snow Crash (Bantam Spectra Book),NEAL STEPHENSON,"In reality, Hiro Protagonist delivers pizza fo..."
3,The Little Red Hen: (Makes a Pizza),Philemon Sturges,"A spirited, quirky, and thoroughly modern rema..."


In [239]:
!pip install git+https://github.com/internetarchive/openlibrary-client.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/internetarchive/openlibrary-client.git to /tmp/pip-req-build-qyaqzof3
  Running command git clone -q https://github.com/internetarchive/openlibrary-client.git /tmp/pip-req-build-qyaqzof3
     |████████████████████████████████| 97 kB 4.9 MB/s 
     |████████████████████████████████| 83 kB 1.8 MB/s 
     |████████████████████████████████| 62 kB 1.2 MB/s 
     |████████████████████████████████| 130 kB 59.1 MB/s 
     |████████████████████████████████| 140 kB 64.2 MB/s 
  Created wheel for openlibrary-client: filename=openlibrary_client-0.0.31-py2.py3-none-any.whl size=42747 sha256=9acb2af933f667aee78cca8d068ae536de179406b4c4a46b8e5543087df5c56a
  Stored in directory: /tmp/pip-ephem-wheel-cache-tljya60z/wheels/c7/99/75/0a92b80fabe41a952d4153d05c840ffc6311835d48d5ee2132
  Created wheel for internetarchive: filename=internetarchive-3.0.2-py3-none-any.whl size=93535 

In [240]:
from olclient.openlibrary import OpenLibrary
ol = OpenLibrary()
author_olid = ol.Author.get_olid_by_name('Dan Brown')
author_obj = ol.get(author_olid)

In [293]:
from IPython.display import display, HTML

In [314]:
def most_similiar_book():
  sentence = [input("Write a line from your favorite book? ")]
  y = vectorizer.transform(sentence)
  cosine_sim = cosine_similarity(X,y)
  max_index = np.argmax(cosine_sim)
  author_olid = ol.Author.get_olid_by_name(df['Author'].iloc[max_index])
  if ol.get(author_olid) != None:
    author_obj = ol.get(author_olid)
  else:
    print('Sorry author is not in our database')
  if author_obj.bio != None:
    author_desc = author_obj.bio.split('\r\n\r\n')[0]
  else:
    author_desc = f"Sorry we dont have any information about {df['Author'].iloc[max_index]} at the moment"
  return print('\n', 'Title:', '\n', df['Title'].iloc[max_index], '\n\n', 'Description:', '\n', df['Blurb'].iloc[max_index], '\n\n', 'About the Author:', '\n', author_desc, 
               display(HTML('<img src="https://covers.openlibrary.org/b/isbn/' + df['ISBN'].iloc[max_index] + '-L.jpg" />')))

In [316]:
most_similiar_book()

Write a line from your favorite book? rats



 Title: 
 Rats (Rats) 

 Description: 
 When a shortcut trail between Pam's backyard to Anna and Lulu's pony paddock is blocked by a neighbor's new sheep pen, the girls fear that Pam will be left out of future Pony Pal excursions. 

 About the Author: 
 Sorry we dont have any information about Paul Zindel at the moment None
